# Load Processed Data into Vector Database

This notebook loads output from data prep kit into Milvus

**Step-4 in this workflow**

![](../media/rag-overview-2.png)


## Configuration

In [1]:

from my_config import MY_CONFIG

## Step-: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.OUTPUT_FOLDER_FINAL)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.OUTPUT_FOLDER_FINAL}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  output-fomc/output_final
Number of parquet files to read :  2

Read file: 'output-fomc/output_final/monetary20240731a1.parquet'.  number of rows = 10
Read file: 'output-fomc/output_final/monetary20240918a1.parquet'.  number of rows = 5

Total number of rows = 15


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   filename          15 non-null     object 
 1   num_pages         15 non-null     int64  
 2   num_tables        15 non-null     int64  
 3   num_doc_elements  15 non-null     int64  
 4   document_id       15 non-null     object 
 5   ext               15 non-null     object 
 6   hash              15 non-null     object 
 7   size              15 non-null     int64  
 8   date_acquired     15 non-null     object 
 9   pdf_convert_time  15 non-null     float64
 10  source_filename   15 non-null     object 
 11  text              15 non-null     object 
 12  doc_jsonpath      15 non-null     object 
 13  page_number       15 non-null     int64  
 14  bbox              15 non-null     object 
 15  chunk_id          15 non-null     int64  
 16  removed           15 no

,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,text,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,vector
0,monetary20240731a1.pdf,4,0,34,a0e52580-4912-4aac-93d4-4fc015b6aa12,pdf,ae2f7f6b109cf8e6388b507c9f866d7946ca907cab6b5a...,11018,2024-09-18T23:55:42.804866,3.573463,monetary20240731a1.pdf,FEDERAL RESERVE press release\nRecent indicato...,$.main-text[3],1,"[71.41007233, 534.21520996, 536.76806641, 630....",0,[],-1,"[0.0007660659, -0.13537231, -0.02113923, 0.091..."
1,monetary20240731a1.pdf,4,0,34,a0e52580-4912-4aac-93d4-4fc015b6aa12,pdf,ae2f7f6b109cf8e6388b507c9f866d7946ca907cab6b5a...,11018,2024-09-18T23:55:42.804866,3.573463,monetary20240731a1.pdf,FEDERAL RESERVE press release\nThe Committee s...,$.main-text[4],1,"[71.60705566, 424.4079895, 538.09204102, 520.4...",1,[],-1,"[0.0034350317, -0.06851143, -0.0934582, 0.0705..."
2,monetary20240731a1.pdf,4,0,34,a0e52580-4912-4aac-93d4-4fc015b6aa12,pdf,ae2f7f6b109cf8e6388b507c9f866d7946ca907cab6b5a...,11018,2024-09-18T23:55:42.804866,3.573463,monetary20240731a1.pdf,FEDERAL RESERVE press release\nIn support of i...,$.main-text[5],1,"[71.49136353, 203.07522583, 541.1159668, 410.0...",2,[],-1,"[0.06745335, -0.039708097, -0.07977984, 0.0748..."


## Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>




In [4]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


# Create A Collection



In [5]:
# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


milvus_client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)


✅ Cleared collection : docs
✅ Created collection : docs


In [6]:
res = milvus_client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

milvus_client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 15


{'row_count': 15}

## Close DB Connection

Close the connection so the lock files are relinquished and other notebooks can access the db

In [7]:
milvus_client.close()

print ("✅ SUCCESS")


✅ SUCCESS


## Test your data by doing a Vector Search

See notebook [vector_search.ipynb](vector_search.ipynb)